# WOfS Validation Accuracy Assessment <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_summary)

## Background
Accuracy assessment for WOfS product in Africa includes generating a confusion error matrix for a WOFL binary classification.
The inputs for the estimating the accuracy of WOfS derived product are a binary classification WOFL layer showing water/non-water and a shapefile containing validation points collected by [Collect Earth Online](https://collect.earth/) tool. Validation points are the ground truth or actual data while the extracted value for each location from WOFL is the predicted value. A confusion error matrix containing overall, producer's and user's accuracy is the output of this analysis. 

## Description
This notebook explains how you can perform accuracy assessment for WOFS derived product using collected ground truth dataset. 

The notebook demonstrates how to:
1. Generating a confusion error matrix for WOFL binary classification
2. Assessing the accuracy of the classification 
***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages
Import Python packages that are used for the analysis.

In [688]:
%matplotlib inline

import time 
import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from rasterio.mask import mask
from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import plot_confusion_matrix, f1_score  
from deafrica_plotting import map_shapefile,display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard,deepcopy
from deafrica_dask import create_local_dask_cluster

### Loading Dataset

Read in the validation data csv, clean the table and rename the column associated with actual and predicted. 

We need to read two columns from this table:
- Water flag as the groundtruth(actual)
- Class Wet from WOfS (prediction)

In [689]:
#Read the ground truth data following analysis step 
CEO = '../Supplementary_data/Validation/Refined/NewAnalysis/Continent/WOfS_processed/Institutions/Point_Based/AEZs/ValidationPoints_Central.csv'
#CEO = '../Supplementary_data/Validation/Refined/NewAnalysis/Continent/WOfS_processed/Institutions/Point_Based/Africa_ValidationPoints.csv'
#CEO ='../Supplementary_data/Validation/Refined/NewAnalysis/Continent/SCL_processed/Institutions/Point_Based/Africa_ValidationPoints.csv'
#CEO = '../Supplementary_data/Validation/Refined/NewAnalysis/Continent/SCL_processed/Institutions/Point_Based/AEZs/ValidationPoints_Eastern.csv'
df = pd.read_csv(CEO,delimiter=",")

In [690]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'PLOT_ID', 'LON', 'LAT',
       'FLAGGED', 'ANALYSES', 'SENTINEL2Y', 'STARTDATE', 'ENDDATE', 'WATER',
       'NO_WATER', 'BAD_IMAGE', 'NOT_SURE', 'CLASS', 'COMMENT', 'MONTH',
       'WATERFLAG', 'geometry', 'CLASS_WET', 'CLEAR_OBS'],
      dtype='object')

In [691]:
#input_data = df.drop(['Unnamed: 0', 'Unnamed: 0.1','Unnamed: 0.1.1','Unnamed__1','FLAGGED', 'ANALYSES','SENTINEL2Y','STARTDATE', 'ENDDATE', 'WATER', 'NO_WATER', 'BAD_IMAGE', 'NOT_SURE','COMMENT','geometry'], axis=1)
input_data = df.drop(['Unnamed: 0', 'Unnamed: 0.1','FLAGGED', 'ANALYSES','SENTINEL2Y','STARTDATE', 'ENDDATE', 'WATER', 'NO_WATER', 'BAD_IMAGE', 'NOT_SURE','COMMENT','geometry'], axis=1)
input_data = input_data.rename(columns={'WATERFLAG':'ACTUAL'})

In [692]:
input_data

,Unnamed: 0.1.1,PLOT_ID,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS
0,2918,137482805.0,16.043110,-9.742657,Open water - freshwater,12,0,0.0,0.0
1,2919,137482805.0,16.043110,-9.742657,Open water - freshwater,12,2,0.0,0.0
2,2920,137482805.0,16.043110,-9.742657,Open water - freshwater,1,1,0.0,0.0
3,2921,137482805.0,16.043110,-9.742657,Open water - freshwater,1,2,0.0,0.0
4,2922,137482805.0,16.043110,-9.742657,Open water - freshwater,2,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
7932,2834,137711866.0,20.938625,-17.973781,Open water - freshwater,4,2,NaN,NaN
7933,2835,137711866.0,20.938625,-17.973781,Open water - freshwater,10,2,1.0,2.0
7934,2836,137711866.0,20.938625,-17.973781,Open water - freshwater,11,2,1.0,2.0
7935,2837,137711866.0,20.938625,-17.973781,Open water - freshwater,12,2,NaN,NaN


In [693]:
countpoints = input_data.groupby('PLOT_ID',as_index=False,sort=False).last()
countpoints

,PLOT_ID,Unnamed: 0.1.1,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS
0,137482805.0,2933,16.043110,-9.742657,Open water - freshwater,11,1,1.0,1.0
1,137482806.0,2953,15.559000,-9.774857,Open water - freshwater,11,2,0.0,0.0
2,137482807.0,2970,28.583027,-9.918961,Open water - freshwater,12,2,1.0,1.0
3,137482808.0,2987,16.323875,-9.971703,Open water - freshwater,12,2,0.0,0.0
4,137482809.0,3004,27.235167,-10.029943,Open water - freshwater,11,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...
495,137711862.0,2790,14.238812,-17.359269,Barren (Bare soil / Rocky Land),10,2,0.0,0.0
496,137711863.0,2802,18.526157,-17.518309,Open water - freshwater,8,2,1.0,1.0
497,137711864.0,2814,19.172259,-17.811881,Open water - freshwater,8,2,0.0,1.0
498,137711865.0,2826,19.847278,-17.880227,Open water - freshwater,12,2,1.0,1.0


In [694]:
#setting the column prediction based on frequency flag or using class_wet flag 
#input_data['PREDICTION'] = input_data['FREQUENCY'].apply(lambda x: '1' if x > 0.5 else '0')
input_data['PREDICTION'] = input_data['CLASS_WET'].apply(lambda x: '1' if x >=1 else '0')  

Remove the Duplicated plot IDs which means those that are labeled for similar month as 0, 1, 2  or 3.

In [695]:
Duplicate = input_data.duplicated(['LAT', 'LON','MONTH'], keep=False)
input_data = input_data[Duplicate==False]

In [696]:
input_data

,Unnamed: 0.1.1,PLOT_ID,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
8,2926,137482805.0,16.043110,-9.742657,Open water - freshwater,4,1,0.0,0.0,0
9,2927,137482805.0,16.043110,-9.742657,Open water - freshwater,5,1,1.0,1.0,1
10,2928,137482805.0,16.043110,-9.742657,Open water - freshwater,6,1,1.0,1.0,1
11,2929,137482805.0,16.043110,-9.742657,Open water - freshwater,7,1,1.0,2.0,1
12,2930,137482805.0,16.043110,-9.742657,Open water - freshwater,8,1,1.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...
7932,2834,137711866.0,20.938625,-17.973781,Open water - freshwater,4,2,NaN,NaN,0
7933,2835,137711866.0,20.938625,-17.973781,Open water - freshwater,10,2,1.0,2.0,1
7934,2836,137711866.0,20.938625,-17.973781,Open water - freshwater,11,2,1.0,2.0,1
7935,2837,137711866.0,20.938625,-17.973781,Open water - freshwater,12,2,NaN,NaN,0


In [697]:
#count the number of duplicates 
count21 = input_data.groupby('PLOT_ID',as_index=False,sort=False).last()
count21

,PLOT_ID,Unnamed: 0.1.1,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,137482805.0,2933,16.043110,-9.742657,Open water - freshwater,11,1,1.0,1.0,1
1,137482806.0,2951,15.559000,-9.774857,Open water - freshwater,10,1,1.0,1.0,0
2,137482807.0,2968,28.583027,-9.918961,Open water - freshwater,11,1,1.0,1.0,0
3,137482808.0,2981,16.323875,-9.971703,Open water - freshwater,9,1,1.0,1.0,1
4,137482809.0,3004,27.235167,-10.029943,Open water - freshwater,11,1,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
136,137711862.0,2790,14.238812,-17.359269,Barren (Bare soil / Rocky Land),10,2,0.0,0.0,0
137,137711863.0,2802,18.526157,-17.518309,Open water - freshwater,8,2,1.0,1.0,1
138,137711864.0,2814,19.172259,-17.811881,Open water - freshwater,8,2,0.0,1.0,0
139,137711865.0,2826,19.847278,-17.880227,Open water - freshwater,12,2,1.0,1.0,1


The following cell is to filter out those rows that are labeled more than 1 or there is no clear WOfS/SCL observations. 

In [698]:
indexNames = input_data[(input_data['ACTUAL'] > 1) | (input_data['CLEAR_OBS']==0.0) | (input_data['CLEAR_OBS'].isna())].index
input_data.drop(indexNames, inplace=True)

In [699]:
count22 = input_data.groupby('PLOT_ID',as_index=False,sort=False).last()
count22

,PLOT_ID,Unnamed: 0.1.1,LON,LAT,CLASS,MONTH,ACTUAL,CLASS_WET,CLEAR_OBS,PREDICTION
0,137482805.0,2933,16.043110,-9.742657,Open water - freshwater,11,1,1.0,1.0,1
1,137482806.0,2946,15.559000,-9.774857,Open water - freshwater,7,1,1.0,1.0,1
2,137482807.0,2966,28.583027,-9.918961,Open water - freshwater,9,1,1.0,1.0,1
3,137482808.0,2981,16.323875,-9.971703,Open water - freshwater,9,1,1.0,1.0,1
4,137482809.0,3002,27.235167,-10.029943,Open water - freshwater,9,1,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...
119,137711862.0,2786,14.238812,-17.359269,Barren (Bare soil / Rocky Land),11,1,1.0,1.0,1
120,137711863.0,2797,18.526157,-17.518309,Open water - freshwater,10,1,1.0,1.0,1
121,137711864.0,2811,19.172259,-17.811881,Open water - freshwater,12,1,1.0,1.0,1
122,137711865.0,2818,19.847278,-17.880227,Open water - freshwater,6,1,1.0,1.0,1


In order to save the table of valid points, the following cell should be run. Otherwise, skip to the next cell.  

In [700]:
input_data.to_csv(('../Supplementary_data/Validation/Refined/NewAnalysis/Continent/WOfS_processed/Institutions/Point_Based/AEZs/ValidPoints/ValidationPoints_Central.csv'))

In [701]:
confusion_matrix = pd.crosstab(input_data['ACTUAL'],input_data['PREDICTION'],rownames=['ACTUAL'],colnames=['PREDICTION'],margins=True)
confusion_matrix

PREDICTION,0,1,All
ACTUAL,,,
0,70,15,85
1,94,352,446
All,164,367,531


`Producer's Accuracy` is the map-maker accuracy showing the probability that a certain class on the ground is classified. Producer's accuracy complements error of omission. 

In [702]:
confusion_matrix["Producer's"] = [confusion_matrix.loc[0][0] / confusion_matrix.loc[0]['All'] * 100, confusion_matrix.loc[1][1] / confusion_matrix.loc[1]['All'] *100, np.nan]
confusion_matrix

PREDICTION,0,1,All,Producer's
ACTUAL,,,,
0,70,15,85,82.352941
1,94,352,446,78.923767
All,164,367,531,NaN


`User's Accuracy` is the map-user accuracy showing how often the class on the map will actually be present on the ground. `User's accuracy` shows the reliability. It is calculated based on the total number of correct classification for a particular class over the total number of classified sites.

In [703]:
#For continent 
users_accuracy = pd.Series([confusion_matrix['0'][0] / confusion_matrix['0']['All'] * 100,
                                confusion_matrix['1'][1] / confusion_matrix['1']['All'] * 100]).rename("User's")

confusion_matrix = confusion_matrix.append(users_accuracy)
confusion_matrix 

,0,1,All,Producer's,0,1
ACTUAL,,,,,,
0,70.0,15.0,85.0,82.352941,NaN,NaN
1,94.0,352.0,446.0,78.923767,NaN,NaN
All,164.0,367.0,531.0,NaN,NaN,NaN
User's,NaN,NaN,NaN,NaN,42.682927,95.912807


`Overal Accuracy` shows what proportion of reference(actual) sites mapped correctly.

In [704]:
confusion_matrix.loc["User's", "Producer's"] = (confusion_matrix['0'][0] + confusion_matrix['1'][1]) / confusion_matrix['All']['All'] * 100
confusion_matrix

,0,1,All,Producer's,0,1
ACTUAL,,,,,,
0,70.0,15.0,85.0,82.352941,NaN,NaN
1,94.0,352.0,446.0,78.923767,NaN,NaN
All,164.0,367.0,531.0,NaN,NaN,NaN
User's,NaN,NaN,NaN,79.472693,42.682927,95.912807


In [705]:
input_data['PREDICTION'] = input_data['PREDICTION'] .astype(str).astype(int)

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1(perfect precision and recall), and is calculated as:

In [706]:
fscore = pd.Series([(2*(confusion_matrix.loc["User's"][0]*confusion_matrix.loc[0]["Producer's"]) / (confusion_matrix.loc["User's"][0] + confusion_matrix.loc[0]["Producer's"])) / 100,
                   f1_score(input_data['ACTUAL'],input_data['PREDICTION'])]).rename("F-score")
confusion_matrix = confusion_matrix.append(fscore)

In [707]:
confusion_matrix

,0,1,All,Producer's,0,1
ACTUAL,,,,,,
0,70.0,15.0,85.0,82.352941,NaN,NaN
1,94.0,352.0,446.0,78.923767,NaN,NaN
All,164.0,367.0,531.0,NaN,NaN,NaN
User's,NaN,NaN,NaN,79.472693,42.682927,95.912807
F-score,NaN,NaN,NaN,NaN,0.562249,0.865929


In [708]:
confusion_matrix = confusion_matrix.round(decimals=2)

In [709]:
confusion_matrix = confusion_matrix.rename(columns={'0':'NoWater','1':'Water', 0:'NoWater',1:'Water','All':'Total'},index={'0':'NoWater','1':'Water',0:'NoWater',1:'Water','All':'Total'})

In [710]:
confusion_matrix

,NoWater,Water,Total,Producer's,NoWater,Water
ACTUAL,,,,,,
NoWater,70.0,15.0,85.0,82.35,NaN,NaN
Water,94.0,352.0,446.0,78.92,NaN,NaN
Total,164.0,367.0,531.0,NaN,NaN,NaN
User's,NaN,NaN,NaN,79.47,42.68,95.91
F-score,NaN,NaN,NaN,NaN,0.56,0.87


In [711]:
#confusion_matrix.to_csv('../Supplementary_data/Validation/Refined/NewAnalysis/Continent/SCL_processed/Institutions/Point_Based/AEZs/ConfusionMatrix/Eastern_confusion_matrix.csv')
confusion_matrix.to_csv('../Supplementary_data/Validation/Refined/NewAnalysis/Continent/WOfS_processed/Institutions/Point_Based/AEZs/ConfusionMatrix/Central_confusion_matrix.csv')

# print(datacube.__version__)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)